# vCAT GLM2
## Create condition-specific EV files
### Task structure: 2 sets/2 runs per set
#### 180 trials/run: *30 BL, *50 cond, *100 fixed

In [ ]:
import os
from os.path import join
import numpy as np
import pandas as pd
from glob import glob

subs = ['sub-005', 'sub-006', 'sub-007', 'sub-008', 'sub-010', 
        'sub-012', 'sub-013', 'sub-014', 'sub-015', 'sub-016', 
        'sub-018', 'sub-019', 'sub-020', 'sub-021', 'sub-022', 
        'sub-023', 'sub-024', 'sub-025', 'sub-026', 'sub-027', 
        'sub-028', 'sub-029', 'sub-030', 'sub-031', 'sub-032']

stim_sets = ['set1', 'set2']

for sub in subs:
    sub_dir = '/home/data/madlab/Mattfeld_vCAT/behav/sub-{0}/'.format(sub)
    dir_files = sorted(glob(join(sub_dir, 'sub-{0}_simp_task*.csv'.format(sub)))) 
    print(sub, len(dir_files))
 
    for i, curr_set in enumerate(stim_sets):
        for curr_run in ['1', '2']:
            if curr_run == '1':
                run = pd.read_csv(dir_files[i*2]) #create dataframe for text files to extract EVS
                #run = run[:-3] #removal of the last 3 trials to avoid scanner artifact
            else:
                run = pd.read_csv(dir_files[i*2+1])
                #run = run[:-3]          
        
            trialtype = run['trialtype'].values #convert dataframes into numpy arrays
            acc = run['acc'].values
            resp = run['resp'].values
            onsets = run['onset'].values
                 
            #grab indices matching multiple specified criteria
            fixed_corr = np.where((acc == 1) & (trialtype == 'fixed'))[0]                                    
            fixed_incorr = np.where((acc == 0) & (trialtype == 'fixed') & (resp != 'None'))[0]       
            cond_corr = np.where((acc == 1) & (trialtype == 'cond'))[0]                             
            cond_incorr = np.where((acc == 0) & (trialtype == 'cond') & (resp != 'None'))[0]                                   
            remaining = np.where((trialtype == 'BL') | (resp == 'None'))[0] #BL and nonresponse trials


            #index onsets array using indices from np.where() criteria
            fixed_corr_onsets = onsets[fixed_corr]
            fixed_incorr_onsets = onsets[fixed_incorr]
            cond_corr_onsets = onsets[cond_corr]
            cond_incorr_onsets = onsets[cond_incorr]
            remaining_onsets = onsets[remaining]

        
            #v-stack matrix containing onsets, durations, and amplitudes in vertical columns for specified trial type (transposed)
            mtrx = np.vstack((onsets, np.ones(len(onsets))*3.0, #Numpy array filled with 3's
                              np.ones(len(onsets)))).T #Numpy array filled with 1's 
            fixed_corr_mtrx = np.vstack((fixed_corr_onsets, np.ones(len(fixed_corr_onsets))*3.0, 
                                         np.ones(len(fixed_corr_onsets)))).T 
            fixed_incorr_mtrx = np.vstack((fixed_incorr_onsets, np.ones(len(fixed_incorr_onsets))*3.0, 
                                           np.ones(len(fixed_incorr_onsets)))).T
            cond_corr_mtrx = np.vstack((cond_corr_onsets, np.ones(len(cond_corr_onsets))*3.0, 
                                        np.ones(len(cond_corr_onsets)))).T
            cond_incorr_mtrx = np.vstack((cond_incorr_onsets, np.ones(len(cond_incorr_onsets))*3.0, 
                                          np.ones(len(cond_incorr_onsets)))).T   
            remaining_mtrx = np.vstack((remaining_onsets, np.ones(len(remaining_onsets))*3.0, 
                                        np.ones(len(remaining_onsets)))).T
         
            if not os.path.exists(join(sub_dir, 'GLM2')): #if directory does not exist
                os.makedirs(join(sub_dir, 'GLM2')) #create it
                
            if curr_run == '1': #if first run in stim set
                np.savetxt(join(sub_dir, 'GLM2', 'run{0}.txt'.format(i*2+1)), mtrx, delimiter='\t', fmt='%.4f')                
                for trial in ['fixed_corr', 'fixed_incorr', 'cond_corr', 'cond_incorr', 'remaining']: #for all trial types
                    exec('np.savetxt(sub_dir+"GLM2/"+"run{0}_{1}.txt",{1}_mtrx,delimiter="\t",fmt="%.4f")'.format(i*2+1,trial))

            else: #if second run in stim set
                np.savetxt(join(sub_dir, 'GLM2', 'run{0}.txt'.format(i*2+2)), mtrx, delimiter='\t', fmt='%.4f')                
                for trial in ['fixed_corr', 'fixed_incorr', 'cond_corr', 'cond_incorr', 'remaining']:
                    exec('np.savetxt(sub_dir+"GLM2/"+"run{0}_{1}.txt",{1}_mtrx,delimiter="\t",fmt="%.4f")'.format(i*2+2,trial))